<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Presidential-Election-Data" data-toc-modified-id="Presidential-Election-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Presidential Election Data</a></span><ul class="toc-item"><li><span><a href="#Online-dataset" data-toc-modified-id="Online-dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Online dataset</a></span></li><li><span><a href="#Web-Scraping" data-toc-modified-id="Web-Scraping-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Web Scraping</a></span></li></ul></li><li><span><a href="#Movie-list-(LGBT-&amp;-Feminism)" data-toc-modified-id="Movie-list-(LGBT-&amp;-Feminism)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Movie list (LGBT &amp; Feminism)</a></span><ul class="toc-item"><li><span><a href="#Web-Scraping" data-toc-modified-id="Web-Scraping-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Web Scraping</a></span></li><li><span><a href="#Online-dataset" data-toc-modified-id="Online-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Online dataset</a></span></li><li><span><a href="#Online-repository" data-toc-modified-id="Online-repository-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Online repository</a></span></li></ul></li><li><span><a href="#Average-Movie-ratings" data-toc-modified-id="Average-Movie-ratings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Average Movie ratings</a></span></li><li><span><a href="#Tweets" data-toc-modified-id="Tweets-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tweets</a></span></li></ul></div>

<div style="text-align:center; font-size:25px; color:#A12A0B"><strong>Popularity of LGBT/Feminist movies by state in US</strong></div> <br>
<div style='text-align:center; font-size:20px'>Open Data Mashups</div>
<div style="text-align:center; font-size:15px"><em>FALL2018</em></div>

In [6]:
# import all necessary packages
import tweepy
import pandas as pd
import requests as req
from lxml import etree
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from tqdm import tqdm_notebook
import csv
import time

## Presidential Election Data
There will be 6 election data from 1996 to 2016 (presidential election is held every 4 years). Data from 2000 to 2016 are in standard format online while data of 1996 needs to perform web scraping. 

- **Data set**: Presidential election data from 1996 to 2016. <br>
- **Goal**: Identify conservative, liberal and swing states. For example, if a state voted for Republic Party more than 3 times during 1996 to 2016, it will be labeled as "Conservative." Since there are 6 election years' data, a state voted for one Party 3 times will be labeled as "Swing."<br>
- **Output**: 2 columns and 52 rows representing states and their corresponding political tendencies. <br><br>
- **Data Source:** 
    1. Existing csv/xls/xlw files online
    2. Web Scraping 1996 election data online

### Online dataset

**Current status**: Standard format files are downloaded. Further step will be conducted to read files into dataframe and identify each state's voting result. <br><br>
Online source: https://transition.fec.gov/pubrec/electionresults.shtml <br>
It contains data of 2000, 2004, 2008, 2012, 2016.

### Web Scraping
**Current status**: Webscraping for 1996 voting result is done as shown below. Further step will be conducted to combine results from 1996 - 2016 to determine each state's political tendency. <br><br>

Web scraping source: https://transition.fec.gov/pubrec/fe1996/elecpop.htm <br>
It contains data from 1996.

In [19]:
# Parse htm file into text content with exception handler
def simple_get(url):
    """
    Attempts to get the content at 'url' by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                # r.text is the content of the response in unicode, 
                # and r.content is the content of the response in bytes.
                return resp.content
            else:
                return None
            
    except RequestException as e:
        log_error('Error during requests to {0}:{1}'.format(url, str(e)))
        return None

In [17]:
# Identify whether the source is in HTML/HTM format or not
def is_good_response(resp):
    """
    Return True if the response seems to be HTML/HTM, Flase otherwise.
    """
    content_type = resp.headers['Content-Type']
    return (resp.status_code == 200 
            and content_type is not None
            and content_type.find('html') > -1)

In [18]:
# Print error message
def log_error(e):
    """
    Print log errors.
    """
    print(e)

In [20]:
# Load 1996 presidential ELECTORAL AND POPULAR VOTE 
url = 'https://transition.fec.gov/pubrec/fe1996/elecpop.htm'
response = simple_get(url)
if response is not None:
    htm = BeautifulSoup(response, 'html.parser')
    # cast to string
    para = str(htm.find_all('pre'))
    temp_content = para[para.find('>AL'):]
    table_content = temp_content[1:temp_content.find('<st')]

In [23]:
# Extract content of election result into a list. Elements in the list represent rows in the raw data 
table_content_li = [x for x in table_content.split('\r\n')]

In [27]:
# Convert to nested list for better processing
# Fill empty space with 'n' indicating 'not voted' for candidates from a specific state
content = []
for row in table_content_li[:-1]:
    a = row.split('        ')
    if a[1] == '':
        a[1] = 'n'
    if a[2] == '':
        a[2] = 'n'
    content.append(a)

In [28]:
# Find exception: one list has different length from the rest due to preprocessing
content[8]

['DC', '3', 'n', '     158,220       17,339', '3,611', '      185,726 ']

In [29]:
# Exception handling
update = content[8][3] + ' ' + content[8][4]
content[8][3] = update

In [30]:
# Check whether the new list meets the requirement
temp_row = content[8]
del temp_row[4]
temp_row

['DC', '3', 'n', '     158,220       17,339 3,611', '      185,726 ']

In [13]:
# replace exception with updated list
content[8] = temp_row

In [14]:
# convert list to numpy array then to dataframe
df96 = pd.DataFrame(content)

In [31]:
# Add column names for dataframe
df96.columns = ['State', 'Clinton','Dole','Popular vote','Total Popular vote']

In [16]:
df96.head(5)

,State,Clinton,Dole,Popular vote,Total Popular vote
0,AL,n,9,"662,165 769,044 92,149","1,534,349"
1,AK,n,3,"80,380 122,746 26,333","241,620"
2,AZ,8,n,"653,288 622,073 112,072","1,404,405"
3,AR,6,n,"475,171 325,416 69,884","884,262"
4,CA,54,n,"5,119,835 3,828,380 697,847","10,019,484"


## Movie list (LGBT & Feminism)
There are three sources to get a comprehensive movie list with LGBT and Feminism themes as shown below. Each source is handled differently based on format. Overlapping exists among movie lists from three sources as data sources are independent from each other. A final list of related movie names will be generated from them.

**Data Source:**<br>
1. Web scraping from webpages:
    * https://en.wikipedia.org/wiki/List_of_LGBT-related_films
    * https://en.wikipedia.org/wiki/Category:Feminist_films
2. Online dataset:
    * https://www.kaggle.com/juzershakir/tmdb-moviesdataset/home
3. Online repository
    * http://files.grouplens.org/datasets/movielens/ml-20m-README.html

### Web Scraping

Wikipedia has lists of LGBT/Feminism topic movies. Since LGBT and Feminism are more of movie topic rather than movie genres like Action, Adventure, Drama, Musical, etc. Movie names on Wikipedia might not be sufficient. That's why further processing on other online dataset is performed.

In [32]:
# LGBT related movie
origin_page = req.get("https://en.wikipedia.org/wiki/List_of_LGBT-related_films")

soup = BeautifulSoup(origin_page.text, "html.parser")

movie_name1 = ''
for element in soup.find_all('a'):
    if element.get('title') is not None:
        movie_name1 += (str(element.string) + "***")

# Get movie names part only
chunks = movie_name1.split('edit***')
for chunk in chunks:
    if chunk.startswith('Z'):
        z_index = chunks.index(chunk)
    if chunk.startswith('$'):
        a_index = chunks.index(chunk)

movie_list1 = chunks[a_index : z_index+1]

# convert each movie into an element of a list
movie_names1 = []
for movie_chunk in movie_list1:
    movie_temp = movie_chunk.split('***')
    movie = movie_temp[:-1]
    movie_names1.extend(movie)
    
# a list of all lgbt movie names from Wiki page
print(movie_names1[0:50])

['$30', 'Boys Life 3', '10 Attitudes', 'The 10 Year Plan', '101 Rent Boys', '101 Reykjavík', '2 × 4', '2 Minutes Later', '2 Seconds', '20 Centimeters', '200 American', '2:37', '24 Nights', 'The 24th Day', '29th and Gay', '3', '3 Dancing Slaves', '3-Day Weekend', '3 Kanya', '30 Years from Here', '4th Man Out', '4.3.2.1', '491', '5ive Girls', '50 Ways of Saying Fabulous', '52 Tuesdays', '54', "'68", '68 Pages', '7 mujeres, 1 homosexual y Carlos', '8 Women', '8: The Mormon Proposition', '9 Dead Gay Guys', "À cause d'un garçon", 'À corps perdu', 'A mi madre le gustan las mujeres', 'À toute vitesse', 'A un dios desconocido', 'Aaron... Albeit a Sex Hero', 'Aban and Khorshid', 'Absent', 'Academy', 'Achilles', 'The Best of Boys in Love', 'Across the Universe', 'AIDS: Doctors and Nurses tell their Stories', 'Ajumma! Are You Krazy???', 'An Act of Valour', 'Adam & Steve', 'Adão e Eva']


In [33]:
# Feminism related movies
origin_page = req.get("https://en.wikipedia.org/wiki/Category:Feminist_films")

soup = BeautifulSoup(origin_page.text, "html.parser")

movie_name2 = ''
for element in soup.find_all('a'):
    if element.get('title') is not None:
        movie_name2 += (str(element.string) + "***")

# Get movie names part only
chunks = movie_name2.split('***')
for chunk in chunks:
#     print(chunk)
    if chunk.startswith('Nor'):
        z_index = chunks.index(chunk)
    if chunk.startswith('5'):
        a_index = chunks.index(chunk)

movie_list2 = chunks[a_index : z_index+1]

# convert each movie into an element of a list
movie_names2 = []
for movie_chunk in movie_list2:
    if movie_chunk.endswith("film)"):
        movie_names2.append(movie_chunk[0:movie_chunk.index("(")-1])
    else:
        movie_names2.append(movie_chunk)
    
# a list of all lgbt movie names from Wiki page
print(movie_names2[0:50])

['5 Girls', '9 to 5', '10 Hours of Walking in NYC as a Woman', '10 Things I Hate About You', '20th Century Women', '22 Female Kottayam', '36 Vayadhinile', 'Aadavantha Deivam', 'Aandhi', 'Aaravalli', 'Aayiram Thalai Vaangi Apoorva Chinthamani', 'The Accused', 'Akka Thangai', 'Aletta Jacobs: Het Hoogste Streven', "Alice Doesn't Live Here Anymore", 'Alice in Wonderland', 'Aliens', 'All About My Mother', 'Anastasia', 'Anatomy of Hell', 'An Angel at My Table', 'Angry Indian Goddesses', 'Anthuleni Katha', "Antonia's Line", 'Arangetram', 'Archana IAS', 'Arth', 'Ask for Jane', 'Assassination Nation', 'The Associate', 'Astitva', 'Attack of the 50 Ft. Woman', 'Aval Appadithan', 'Aval Oru Thodar Kathai', 'Avargal', 'Bad Girls', 'Bad Moms', 'Bagdad Cafe', 'The Ballad of Josie', 'The Ballad of Little Jo', 'Bandit Queen', 'Barb Wire', 'Basic Instinct', 'Battle of the Sexes', 'Becoming Jane', 'Bed and Sofa', 'The Beguiled', 'Big Eyes', 'Bol', 'Born in Flames']


### Online dataset
A comprehensive csv data file from kaggle.com with movie names, keywords, budgets, revenue, etc. Keywords indicate the theme and category of the film, which will be used to find LGBT/Feminism movie names.
<br><br>
**Url**: https://www.kaggle.com/juzershakir/tmdb-moviesdataset/home

In [34]:
file = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating2/tmdb_movies_data.csv')

# keywords for search
searchfor = ['lgbt','gay','lesb','strong woman','strong women','femin','homo']

# in order to search multiple keywords at a time
file_alter = file[file['keywords'].str.contains('|'.join(searchfor), na=False)]

In [35]:
# Get 196 rows of data: 196 movie names
file_alter.shape

(196, 21)

In [44]:
file_alter['original_title'].head(10)

19     The Hunger Games: Mockingjay - Part 2
50                                     Carol
157                      Ricki and the Flash
168                              Suffragette
200                                 Freeheld
243                            Dirty Weekend
300                     The Duke of Burgundy
364                            The Girl King
521                     Appropriate Behavior
538                     The Mask You Live In
Name: original_title, dtype: object

### Online repository

In [ ]:
file_rating = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/ratings.csv')
file_tag = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/tags.csv')

In [ ]:
file_rating.head()

In [ ]:
file_tag.head()

In [ ]:
file_movie = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/movies.csv')

In [ ]:
file_movie.head()

In [ ]:
file_link=pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/links.csv')
file_link.head()

In [ ]:
# search keywords from "tag" for lgbt/feminist related movies
name_tag = file_tag.merge(file_movie, on='movieId')
name_tag_alter = name_tag[name_tag['tag'].str.contains('|'.join(searchfor), na=False)]


In [ ]:
name_tag_alter.head(10)


## Average Movie ratings
**Data Source:** <br>
- Online repository:
    http://files.grouplens.org/datasets/movielens/ml-20m-README.html

In [ ]:
# This contains multiple users' ratings on the same movies
movie_rating = name_tag_alter.merge(file_rating, on=['userId','movieId'])


In [ ]:
movie_rating.head(10)

In [ ]:
# Calculate average movie ratings
movieid_rating = movie_rating.groupby('movieId')[['rating']].mean()
movieid_rating=movieid_rating.reset_index()

In [ ]:
moviename_rating = movieid_rating.merge(file_movie, on='movieId')
moviename_rating.head(10)

In [ ]:
moviename_rating.shape

## Tweets
Using Twitter API to get people's reviews on movies.

In [ ]:
# authorization


auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Create tweepy object for twitter API
api = tweepy.API(auth)

In [ ]:
# Read from Twitter ID file
tweet_summary_map = {}
nameList = ['being julia', 'brokeback mountain']
for movie in nameList:
    with open("Tweet Data/Tweepy-API-xPath/"+movie+".txt", 'r') as f:
        x = f.read().splitlines()
    id_list = [line.split('/')[-1] for line in x]
    tweet_summary = pd.DataFrame(columns=['Timezone', 'Full Tweet', 'user_name', 'user_location', 'coordinates', 'country_code', 'place'])
    tweet_summary.index.name = 'Tweet Time'
    for id in tqdm_notebook(id_list):
        try:
            tweet_info = api.get_status(id, lang = 'en', tweet_mode='extended')
            if 'retweeted_status' in dir(tweet_info):
                tweet=tweet_info.retweeted_status.full_text
            else:
                tweet=tweet_info.full_text
            if tweet_info.place:
                place = tweet_info.place.full_name
                country_code = tweet_info.place.country_code
            else:
                place = None
                country_code = None
        except:
            pass
        tweet_summary.loc[tweet_info.created_at] = [tweet_info.user.time_zone, tweet, tweet_info.user.name, tweet_info.user.location, tweet_info.coordinates, country_code, place]
    tweet_summary_map[movie] = tweet_summary
    time.sleep(1)

In [ ]:
tweet_summary_map['brokeback mountain']

In [ ]:
tweet_summary_map['being julia']
